In [1]:
import random

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision.models.resnet import resnet50
import torchvision.transforms as transforms

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
!nvidia-smi

Mon Nov 20 13:55:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install torch-lr-finder

In [5]:
train_transforms = torchvision.transforms.Compose([
        torchvision.transforms.ColorJitter(brightness=0.1,contrast=0.1,saturation=0.1),
        torchvision.transforms.RandomAffine(15),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomRotation(15),
        torchvision.transforms.Resize((224,224)),
        torchvision.transforms.ToTensor(),
        #torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
valid_transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224)),
        torchvision.transforms.ToTensor(),
        #torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),##imagenet mean and std
])

In [7]:
train_comp_dataset=torchvision.datasets.ImageFolder(root='/kaggle/input/plant-based-meat-products/Train',transform=train_transforms)

In [8]:
num_train = len(train_comp_dataset)
indices = list(range(num_train))

num_train

6160

In [9]:
train_comp_dataset.classes


['commercial_deep_normal',
 'commercial_deep_over',
 'commercial_unbaked',
 'inhouse_deep_normal',
 'inhouse_deep_over',
 'inhouse_old_deep_normal',
 'inhouse_old_deep_over',
 'inhouse_unbaked']

In [10]:
random.seed(42)
random.shuffle(indices)

In [11]:
dataset_size = num_train
train_ratio = 0.8
val_ratio = 0.2

train_size = int(train_ratio * dataset_size)
val_size = int(val_ratio * dataset_size)
test_size = dataset_size - train_size - val_size

In [12]:
print(train_size)
print(val_size)
print(test_size)

4928
1232
0


In [13]:
train_indices = indices[:train_size]
val_indices = indices[train_size: train_size + val_size]
test_indices = indices[train_size + val_size : ]

In [14]:
train_dataset = torch.utils.data.Subset(train_comp_dataset, train_indices)
valid_dataset = torch.utils.data.Subset(train_comp_dataset, val_indices)

In [15]:
batch_size = 16
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size,shuffle=True,num_workers=2,pin_memory=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset,batch_size,shuffle=False,num_workers=2,pin_memory=True)

In [16]:
train_loader

In [17]:

model=torch.load('/kaggle/input/resnet-50/resnet50_experiment.pt')

In [18]:
##For the colour models
model.fc = torch.nn.Sequential(
    torch.nn.Linear(2048,512),
    torch.nn.Linear(512,128),
    torch.nn.Linear(128,32),
    torch.nn.Linear(32,14),
    #torch.nn.Linear(8,2)
)

In [19]:
from torch_lr_finder import LRFinder
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
#lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
lr_finder = LRFinder(model, optimizer, criterion)
lr_finder.range_test(train_loader,end_lr=0.05, num_iter=10)
lr_finder.reset()


/opt/conda/lib/python3.10/site-packages/torch_lr_finder/lr_finder.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/10 [00:00<?, ?it/s]

Learning rate search finished. See the graph with {finder_name}.plot()


In [20]:
train_acc_list = []
val_acc_list = []
train_loss_list = []
val_loss_list = []

In [21]:
cuda = True
epochs = 25
model_name = './resnet50_experiment.pt'
optimizer = torch.optim.Adam(model.parameters(),lr=1.00E-03,weight_decay=0.001)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.1,patience=1,verbose=True)

writer = SummaryWriter() # For Tensorboard
early_stop_count=0
ES_patience=6
best = 0.0
if cuda:
    model.cuda()

for epoch in range(epochs):
    
    # Training
    model.train()
    correct = 0
    train_loss = 0.0
    tbar = tqdm(train_loader, desc = 'Training', position=0, leave=True)
    for i,(inp,lbl) in enumerate(tbar):
        optimizer.zero_grad()
        if cuda:
            inp,lbl = inp.cuda(),lbl.cuda()
        out = model(inp)
        loss = criterion(out,lbl)
        train_loss += loss
        out = out.argmax(dim=1)
        correct += (out == lbl).sum().item()
        loss.backward()
        optimizer.step()
        tbar.set_description(f"Epoch: {epoch+1}, loss: {loss.item():.5f}, acc: {100.0*correct/((i+1)*train_loader.batch_size):.4f}%")
    train_acc = 100.0*correct/len(train_loader.dataset)
    train_loss /= (len(train_loader.dataset)/batch_size)

    # Validation
    model.eval()
    with torch.no_grad():
        correct = 0
        val_loss = 0.0
        vbar = tqdm(valid_loader, desc = 'Validation', position=0, leave=True)
        for i,(inp,lbl) in enumerate(vbar):
            if cuda:
                inp,lbl = inp.cuda(),lbl.cuda()
            out = model(inp)
            val_loss += criterion(out,lbl)
            out = out.argmax(dim=1)
            correct += (out == lbl).sum().item()
        val_acc = 100.0*correct/len(valid_loader.dataset)
        val_loss /= (len(valid_loader.dataset)/batch_size)
    print(f'\nEpoch: {epoch+1}/{epochs}')
    print(f'Train loss: {train_loss}, Train Accuracy: {train_acc}')
    print(f'Validation loss: {val_loss}, Validation Accuracy: {val_acc}\n')

    scheduler.step(val_loss)

    # write to tensorboard
    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Loss/val", val_loss, epoch)
    writer.add_scalar("Accuracy/train", train_acc, epoch)
    writer.add_scalar("Accuracy/val", val_acc, epoch)
    
     # Append the accuracy and loss values to the respective lists
    train_acc_list.append(train_acc)
    val_acc_list.append(val_acc)
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)
    

    if val_acc>best:
        best=val_acc
        torch.save(model,model_name)
        early_stop_count=0
        print('Accuracy Improved, model saved.\n')
    else:
        early_stop_count+=1
        
#     if val_acc > 95:
#         best=val_acc
#         torch.save(model,model_name)
#         print(f'Accuracy achieved: {best:.2f}%. Stoppping Early to prevent overtraining.')
#         break

    if early_stop_count==ES_patience:
        print('Early Stopping Initiated...')
        print(f'Best Accuracy achieved: {best:.2f}% at epoch:{epoch-ES_patience}')
        print(f'Model saved as  {model_name}')
        break
    writer.flush()
# writer.close()

Validation: 100%|██████████| 77/77 [00:07<00:00, 10.31it/s]



Epoch: 1/25
Train loss: 0.7660456299781799, Train Accuracy: 70.19074675324676
Validation loss: 0.48559996485710144, Validation Accuracy: 79.46428571428571

Accuracy Improved, model saved.



Validation: 100%|██████████| 77/77 [00:05<00:00, 15.35it/s]



Epoch: 2/25
Train loss: 0.4067362844944, Train Accuracy: 84.8823051948052
Validation loss: 0.46010053157806396, Validation Accuracy: 79.38311688311688



Validation: 100%|██████████| 77/77 [00:04<00:00, 16.65it/s]


...


Epoch: 21/25
Train loss: 0.09024760872125626, Train Accuracy: 96.75324675324676
Validation loss: 0.10817557573318481, Validation Accuracy: 96.18506493506493

Epoch 00021: reducing learning rate of group 0 to 1.0000e-08.


Validation: 100%|██████████| 77/77 [00:04<00:00, 16.40it/s]



Epoch: 22/25
Train loss: 0.09545815736055374, Train Accuracy: 96.6112012987013
Validation loss: 0.10152868181467056, Validation Accuracy: 96.02272727272727



Validation: 100%|██████████| 77/77 [00:04<00:00, 17.06it/s]


Epoch: 23/25
Train loss: 0.08990858495235443, Train Accuracy: 96.77353896103897
Validation loss: 0.09311787784099579, Validation Accuracy: 96.42857142857143

Early Stopping Initiated...
Best Accuracy achieved: 97.16% at epoch:16
Model saved as  ./resnet50_experiment.pt


In [22]:
train_loss_list = list(map(lambda x: x.item(), train_loss_list))
val_loss_list = list(map(lambda x: x.item(), val_loss_list))

In [23]:
print(train_acc_list)
print(val_acc_list)
print(train_loss_list)
print(val_loss_list)

[70.19074675324676, 84.8823051948052, 86.9926948051948, 88.41314935064935, 89.0625, 89.71185064935065, 89.89448051948052, 91.19318181818181, 94.62256493506493, 95.08928571428571, 95.23133116883118, 95.65746753246754, 96.1038961038961, 96.53003246753246, 96.875, 96.46915584415585, 96.65178571428571, 96.48944805194805, 97.34172077922078, 96.93587662337663, 96.75324675324676, 96.6112012987013, 96.77353896103897]
[79.46428571428571, 79.38311688311688, 89.6103896103896, 87.17532467532467, 91.88311688311688, 92.53246753246754, 89.28571428571429, 90.3409090909091, 95.53571428571429, 96.5909090909091, 95.94155844155844, 96.75324675324676, 96.02272727272727, 95.69805194805195, 96.99675324675324, 96.26623376623377, 97.1590909090909, 96.83441558441558, 96.5909090909091, 96.5909090909091, 96.18506493506493, 96.02272727272727, 96.42857142857143]
[0.7660456299781799, 0.4067362844944, 0.356037974357605, 0.3029771149158478, 0.29542601108551025, 0.2834942936897278, 0.2672704756259918, 0.233774304389953

In [24]:
pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 16.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [26]:
import plotly.graph_objs as go
import plotly.io as pio

# Plotting accuracy and loss graphs
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, len(train_acc_list)+1)), y=train_acc_list, mode='lines', name='Train Accuracy'))
fig.add_trace(go.Scatter(x=list(range(1, len(val_acc_list)+1)), y=val_acc_list, mode='lines', name='Validation Accuracy'))
fig.update_layout(title='Accuracy Graph', xaxis_title='Epochs', yaxis_title='Accuracy')
fig.show()
pio.write_image(fig, '/kaggle/working/accuracy_plot.png')

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, len(train_loss_list)+1)), y=train_loss_list, mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=list(range(1, len(val_loss_list)+1)), y=val_loss_list, mode='lines', name='Validation Loss'))
fig.update_layout(title='Loss Graph', xaxis_title='Epochs', yaxis_title='Loss')
fig.show()
pio.write_image(fig, '/kaggle/working/loss_plot.png')

In [87]:
if torch.cuda.is_available():
    device = torch.device('cuda')  # Use GPU

else:
    device = torch.device('cpu')   # Use CPU

model = torch.load('./resnet50_experiment.pt', map_location=device)

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    #         torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])